# Foundations of Data Science
## S1 Week 05: Data wrangling II - groupby and regular expressions

**Learning outcomes:** 
In this lab, we will build on what you have previously learned on preparing data for future use. By the end of the lab you should be able to:

- use regular expressions to parse textual data
- organise columns into indicator variables when appropriate
- apply group-wise computations to your data

In this week's lab, you will be analysing data from the course survey from last year, as well as data obtained from the department of Informatics timetabling. There are many instances whereby you might want to create a form and analyse user responses to it - let this be a lesson in how (not?) to design a form 🤔.

**Data description:** The data from the first dataset for this lab originates from last year's course optional survey, which included several questions given to past students inquiring about their remote working setup, Python expertise and interests. The data was anonymised prior to being provided for this lab. The second dataset is the Informatics timetables which can be obtained from the web.

In [ ]:
import os
import re

import pandas as pd
import numpy as np
import seaborn as sns
# Package to display the hints and soultions
from common.show_solutions import show

## A. Exploring the survey data

### A.1 Loading and preparing the survey data

**Exercise 01:** Load `survey_data.xlsx` into a variable `survey_data`.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=1)

In [ ]:
# Your code

Whenever working with data, remember it's important to first get an idea of what your data looks like. Let's start by having a look at the dtypes of the columns.

In [ ]:
survey_data.info()

With the exception of 'ID', pandas has inferred the types of the remaining columns as Python 'objects' (text or mixed numeric and non-numeric values). In particular, notice that we have a column named as though it should contain a timestamp, '...what is the time in my current location?'. We'll look at converting this to a proper time format.

But first, you might (not!) have noticed the double space typo in the second column name. Let's tidy up some of the names so they're easier to reference later. Using [`pandas.DataFrame.rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.htmlhttps://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html), we can specify the names of existing columns and what they should be renamed to.

In [ ]:
survey_data = survey_data.rename(columns={survey_data.columns[1]: 'Preferred tutorial loc', # This is equivalent to {'My  preference for tutorial location is': 'Preferred tutorial loc', ...
                                          survey_data.columns[2]: 'Time at 10:00 BST',
                                          survey_data.columns[3]: 'Python expertise',
                                          survey_data.columns[4]: 'IT equipment',
                                          survey_data.columns[5]: 'Interests'})
print("Column names:", list(survey_data.columns))

Although date and time are not a data type of their own in Python, the [Datetime](https://docs.python.org/3/library/datetime.html#module-datetime) module supplies classes and functions to work with them. Representing textual time columns as Datetime objects can be handy for querying rows that meet specific time conditions and performing simple date and time arithmetic. The pandas equivalent to the Python Datetime object is [Timestamp](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html).

### A.2 Converting textual time data to a datetime representation

One option is to use [`pd.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to convert the string values in a column to a Timestamp if the string meets a format of our choice (specified using [strftime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) date format codes, e.g. `%m/%d/%Y` to match with `12/19/1995`). This function has the pandas `errors` argument that we've seen before in lab 2 (in `pd.to_numeric`), allowing us to `raise` an error if a value doesn't match the desired format, `coerce` it to be set as `NaT` or `ignore` it to return it as it is.

If the values in our time column don't follow a rigid string format already, we might end up losing a lot of our data.

**Exercise 02:** Print out the _unique_ values in the 'Time at 10:00 BST' column and their corresponding count in the column.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=2)

In [ ]:
# Your code

**Discussion 01:** Did you expect to see so many variations in the format of the values in the time column? Discuss with your partner the implications of this on using the function [`pd.to_datetime()`](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to convert the column to a datetime representation. What [`strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) format could you use to successfully convert most of the values?

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=2.1)

Your answer:

How else could we clean up more of the textual time column prior to attempting to convert it a useful datetime type?

### Regular expressions

You may have encountered [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) before, which are also referred to as __regexs__. These are sets of character combinations that Python (and many other programming languages) can understand and use as rules to search through character strings.

You can use these regex rules to ask questions such as “Does this string match the pattern?”, or “Is there a match for the pattern anywhere in this string?”. You can also use regexs to modify or split apart a string in various ways.

We can use the Python [`re`](https://docs.python.org/3/library/re.html) package for this. Some simple examples are shown in the cell below.

In [ ]:
# Simple example: a pattern that matches with the first name Sarah or Sara
pattern = '^Sara(h?)$'
# ^ matches with the start
# () contains an expression to capture and group
# ? zero or one occurrence
# $ matches with the end

# re.search looks for any location where the regex matches; re.match checks only the beginning.
# If successful, it returns a Match object with info such as where the match starts and ends,
# the substring it matched, and more. If no match is found, None is returned.
print(re.search(pattern, 'Sarah'))
print(re.search(pattern, 'Sara'))
print(re.search(pattern, 'sarah'))

# re.findall returns all substrings where the regex matches as a list
eg_sentence = 'Sarah went to the park. Sarah wished it would rain.'
print(re.findall('Sara(h?)', eg_sentence))
print(re.findall('Sara(?:h)', eg_sentence))
# The last statement uses (?:...), a non-capturing version of regular parentheses.
# It matches whatever regex is inside the parentheses, but the substring matched by the group cannot
# be retrieved after performing a match or referenced later in the pattern (re.findall otherwise returns the matched group only).

Several [methods](https://docs.python.org/3/library/re.html#match-objects) can be called on the 'Match' object that's returned to us. One useful method is `group()` which returns one or more string subgroups of the match. By default it returns the entire match, but if we pass an integer (up to 100) the substring matching the corresponding parenthesised group is returned.

**Excerise 03:** Can you consult the Python [regex syntax](https://docs.python.org/3/library/re.html) documentation to construct a general regular expression pattern to extract the string 'bulldog' from the string 'data/CIFAR-10/images/bulldog_146.jpg'.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=3)

In [ ]:
search_str = 'data/CIFAR-10/images/bulldog_146.png'
# your code

So, back to cleaning up the time data column. We can use regular expressions to define a set of rules that progressively capture the variations in the format of strings in the 'Time at 10:00 BST' column, and substitute the matched sub-strings appropriately to convert them to a consistent string format.

**Exercise 04:** In the cell below, `time_clean()` is a stub of a function that cleans the times. We've done some of the more tricky replacements for you, and your task is to try making some of the other replacements.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=4)

In [ ]:
def time_clean(t):
    """Clean a string from the survey data to a consistent string format ('%H:%M').
    This function should handle the string cases found in Exercise 02."""
    if isinstance(t, str):  # NaN values will be of type float
        # re.sub replaces the parts of a string matching a pattern with a desired replacement
        t = re.sub('：', ':', t) # Replace "fullwidth colon" (Unicode character U+FF1A) with standard colon
        t = re.sub(' +', '', t) # Remove spaces in the middle of the string
        if re.match('^([^:]+)[Pp][Mm]', t): # 5pm -> 17:00
            hour = re.sub('^([^:]+)[Pp][Mm]', '\\1', t)
            t = str((int(hour) + 12)) + ":00"
        # Your answer:    
        # More lines in the format
        # t = re.sub...
        
    
    return(t)

# Example string to clean
example_time = "10:00(10am) "
print(example_time, "->", time_clean(example_time))

**Exercise 05:** Use `time_clean()` to convert each value in the 'Time at 10:00 BST' column to a pandas Timestamp format.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=5)

In [ ]:
# Your code

Now that our column is represented as a datetime, we can perform time-related queries on it such as,

In [ ]:
after_midday = survey_data['Time at 10:00 BST'] >= "12:00"
pm_df = survey_data.loc[after_midday]
print("Number of users in the 'pm' during the lecture:", pm_df.shape[0])

**Exercise 06:** Query the data to determine:

a) the earliest time the lecture takes place for any of the students.

b) the number of users that are local to UK time.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=6)

In [ ]:
# Your code

### A.3 Dummy / Indicator variables

The survey data has several columns that contain categorical entries, and users can have multiple categorical options for 'IT equipment' and 'Interests' which are currently separated by semicolons. 

Depending on the analysis you might want to perform, it can be beneficial and neccessary to create dummy or indicator variables in order to handle numerical computations on your data. These are binary variables that indicates whether a separate categorical variable takes on a specific value for an entry.

[`pd.Series.str.get_dummies()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html) is a function we can use to convert our categorical variable into dummy/indicator variables. We use dummy variables when applying regression and classification algorithms to categorical data. It also has a handy argument for specifying a separator on which to split our multiple entries by. After converting to this format, the presence of the category receives a value of 1 and in its absence the value is 0.

In [ ]:
equipment_expanded = survey_data['IT equipment'].str.get_dummies(sep=";")
equipment_expanded.head()

In [ ]:
survey_equip_expanded = pd.concat([survey_data, equipment_expanded], axis=1)
survey_equip_expanded.head()

**Exercise 07 (optional):** You will see that some of the column headers are very long. This is because, along with a categorical list of 'IT equipment', users could also specify free-written responses to contain extra details. Write a regular expression that removes any sub-entry specified which is longer than three words and apply it to the 'IT equipment' column. Make sure to keep an eye out for NaN values and whitespace. Then repeat the conversion above to dummy variables.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=7)

In [ ]:
# Your code

## B. School of Informatics timetabling

### B.1 Loading the Informatics timetables

The timetables for courses in the School of Informatics can be found online (e.g. see ours [here](https://www.ted.is.ed.ac.uk/UOE2021_SWS/TIMETABLE.ASP?OBJECTCLASS=MODULE&IDENTIFIER=INFR08030_SS1_YR&STYLE=TEXTSPREADSHEET&TEMPLATE=SWSCUST+OBJECT+TEXTSPREADSHEET&WEEK=9-37)). We downloaded the webpage specifying the timetables for all of the courses within the School of Informatics for Semester 1 in 2021-22; see `SWSCUST Object TextSpreadsheet.html` in the `datasets` folder. The timetables are written in Hypertext Markup Language (HTML), a language which defines the meaning and structure of web content. It's exactly what you would see if you looked at the page source in the browser.

We will learn more about the details of web-scraping in a later lab. Today we will focus on using pandas to read a HTML file and performing group-wise computations across the data.

One option for specifically parsing HTML tables is to use [`pd.read_html()`](https://pandas.pydata.org/docs/reference/api/pandas.read_html.html). This function searches for the `<table>` elements in the HTML page and returns them as a list. 

We advise opening 'SWSCUST Object TextSpreadsheet.html' in your web-browser to have a better look at it. You might notice that many of the tables are empty, under certain days of the week for example. We can use the `match` argument to only return tables that contain text matching a string or regular expression that we specify. The value defaults to `.+` (i.e. matches any non-empty string).

In [ ]:
timetable_data_path = os.path.join(os.getcwd(), 'datasets', 'SWSCUST Object TextSpreadsheet.html')
timetable_data_list = pd.read_html(timetable_data_path, match='Activity', header=0)
print("Tables parsed:", len(timetable_data_list))
print("Example table:")
timetable_data_list[2].head()

**Exercise 08:** We currently have a list of DataFrames, where each DataFrame is a table parsed from the HTML page. Can you create one DataFrame that contains all of these tables? Assign it the variable name `timetable_data`.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=8)

In [ ]:
# Your code

In [ ]:
timetable_data.shape

Regular expressions can also be useful for stripping extra whitespace or unnecessary characters from your data.

**Exercise 09:** Use a regular expression to strip the '*' characters from the values in the 'Type' column.

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=9)

In [ ]:
# Your code

Since `pd.read_html` only parses `<table>` elements, with the layout of 'SWSCUST Object TextSpreadsheet.html' its tricky to neatly associate each table with its associated day of the week (which are specified in `<p>` tags) and course. HTML can be notoriously messy to parse. Thankfully, there are several external libraries which can be useful for this task, some of which we'll look at in a later lab.

The current 'Activity' column of our data contains the name of the course, and sometimes specific details regarding the activity. For now, we'll try to remove the latter details so we can later group our timetables by course.

In [ ]:
def course_name_from_activity(activity_str):
    """Attempts to extract the course name from the 'Activity' column
    of the timetable data. Warning: this function is not perfect! There are still some
    exceptions in the formatting of the course names which are not captured."""
    if re.search('[-]', activity_str):
        match = re.match('^(Informatics \d - )?.*(?= - )', activity_str)
        assert match is not None, f"No match found for '{activity_str}'."
        return match.group()  # Return the matched string (instead of Match object)
    return activity_str

timetable_data["Course Name"] = timetable_data["Activity"].apply(lambda x: course_name_from_activity(x))

num_activities = timetable_data["Activity"].nunique()
extracted_courses = timetable_data["Course Name"].nunique()
print(f"{num_activities} detailed activities reduced to {extracted_courses}.")

### B.2 Groupwise computations

By _groupby_ (or _split-apply-combine_) we are referring to a process involving one or more of the following steps:

* Splitting the data into groups based on some criteria.
* Applying a function to each group independently. This could include computing a summary statistic for each group, such as a mean or count; performing some group-specific computation such as standardising the data within a group; as well as filtering to discard data that has only a few members, for example.
* Combining the results into a data structure.

In [ ]:
# First let's drop some of the empty or redundant columns
timetable_data = timetable_data.drop(['Activity', 'Description', 'Staff'], axis=1)
timetable_data.head()

To group our data according to course name, we can do the following:

In [ ]:
# Group data by variable of interest
grouped_by_name = timetable_data.groupby("Course Name")
# Retrieve a given group by name
grouped_by_name.get_group("Accelerated Natural Language Processing")

Computations can then be performed on the grouped DataFrame like finding the size of each group:

In [ ]:
# Count the number of 'activities' scheduled for each course in semester 1
grouped_by_name.size()

We can check that a few of the groups have the size that we expect in the list above:

In [ ]:
print(len(grouped_by_name.get_group("Accelerated Natural Language Processing")))
print(len(grouped_by_name.get_group("Natural Computing")))

We can also group by multiple variables:

In [ ]:
# For each course, display the number of scheduled entries for each activity type
timetable_data.groupby(["Course Name", "Type"]).size()

You can apply various functions to `GroupBy` objects. For example `.mean()` and `.median()` compute the mean and median of the numeric variables in each group.

**Exercise 10:** Ignoring entries which have NaN values, perform group-wise computations on the timetable data (and refer to pandas documentation) to answer the following questions:

a) Which building has the largest number of rooms in it? 

b) Which room is used the most across all the buildings, and which building is it in?

c) What is the mean and median length of a 'Lecture' in Informatics?

d) Which on average, is the longest activity type scheduled (i.e. lectures, tutorials, labs, etc.)?

In [ ]:
# Run this cell to be offered with hints and solution
show(week=5, question=10)

In [ ]:
# Your code

In [ ]:
# Your code

In [ ]:
# Your code

In [ ]:
# Your code